Make artificial negatives from random sampling on KG.
* Min distance from target 3.
    * https://networkx.org/documentation/stable/reference/generated/networkx.generators.ego.ego_graph.html
    * `ego_graph` 
* Opt: maybe sample  from common CUIs or terms? 

In [1]:
import warnings
warnings.filterwarnings("ignore")
# warnings.filterwarnings("W036")
import logging
logger = logging.getLogger("spacy")
logger.setLevel(logging.ERROR)
import pandas as pd

import json
import re
from tqdm import tqdm

import pandas as pd

import json
# from pydantic import BaseModel, Field, ValidationError
# from typing import Optional, List
import time

# from ukb_target_pipe  import  make_target_df,model_features,ipw_downsampling#*
# from search_pubmed import  run_search_pubmed #*
from configs import * #config_gall

from util import  *  #generate_boring_prompts
from Link_semmed_cuis import  * #link_kg_concepts #*
# from Link_semmed_cuis import  link_kg_concepts, get_kg_connections



In [2]:
FAST = True

## import from configs
all_configs = [
    #     config_celiac, config_gall,config_gout, 
    # config_spine, config_oesophagus,config_heart,
    # config_eye_occ,
    config_depression
              ]
if FAST:
    all_configs = [config_eye_occ]
config = all_configs[0]

input_kg_path="../../SemMed/predications.parquet"

In [3]:
config

{'TARGET_CODES_LIST': ('H34',),
 'do_IPW': True,
 'do_boruta_fs': False,
 'K_IPW_RATIO': 9,
 'targets': ['Retinal Vein Occlusion', 'retinal artery occlusion', 'CRVO'],
 'FEATURES_REPORT_PATH': 'eye_occ_feature_report.csv',
 'QUERY_CANDIDATES_FILE': 'candidate_novel_cuis_eye_occ.csv',
 'CANDIDATE_NOVEL_CUIS_FILEPATH': 'candidate_novel_cuis_eye_occ.csv',
 'TARGET_NAME': 'Retinal Vein Occlusion',
 'additional_target_cui_terms_list': [],
 'OUTPUT_RES_PREFIX': 'eye_occ_'}

In [4]:
df_kg = pd.read_parquet(input_kg_path)

df_kg = df_kg.loc[df_kg["pair_counts"] >= 3].reset_index(drop=True).copy()
df_kg.drop(columns=["pair_counts","counts"], inplace=True, errors="ignore")
df_kg

,SUBJECT_CUI,SUBJECT_NAME,PREDICATE,OBJECT_CUI,OBJECT_NAME,first_year_pair,first_year_triple
0,C0087111,Therapeutic procedure,TREATS,C0030705,Patients,1902,1902
1,C0040395,tomography,USES,C1441526,COMPUTED,1956,1956
2,C0012634,Disease,PROCESS_OF,C0030705,Patients,1902,1902
3,C0543467,Operative Surgical Procedures,TREATS,C0030705,Patients,1821,1901
4,C1457887,Symptoms,PROCESS_OF,C0030705,Patients,1898,1898
...,...,...,...,...,...,...,...
9139031,100,ADA,AFFECTS,C0011847,Diabetes,1994,1994
9139032,100,ADA,AFFECTS,C0002074,Alkylation,1988,1988
9139033,1,A1BG,higher_than,1,A1BG,2008,2017
9139034,1,A1BG,AUGMENTS,C0596290,Cell Proliferation,2014,2017


In [5]:
df_kg["SUBJECT_NAME"].value_counts().head(20)

SUBJECT_NAME
Therapeutic procedure            57403
Cells                            49148
Pharmaceutical Preparations      40097
Proteins                         35548
Genes                            29832
Operative Surgical Procedures    20868
Disease                          20627
Antibodies                       20133
Patients                         19810
Body tissue                      19386
Enzymes                          17283
DNA                              16455
Mutation                         16325
Insulin                          15774
Amino Acids                      14726
cytokine                         14458
Glucose                          14194
receptor                         13673
Assay                            13556
Brain                            13443
Name: count, dtype: int64

In [6]:
if FAST:
    nlp = spacy.load("en_core_sci_sm")
else:
    nlp = spacy.load("en_core_sci_lg")
    # nlp = spacy.load("en_core_sci_scibert") # error in later spacy version?

# This line takes a while, because we have to download ~1GB of data
# and load a large JSON file (the knowledge base). 

# Add the abbreviation pipe to the spacy pipeline. (if using resolve_abbreviations )
nlp.add_pipe("abbreviation_detector")
nlp.add_pipe("scispacy_linker",
             config={"resolve_abbreviations": True,
                     "linker_name": "umls",
                     "max_entities_per_mention": 3  # 3, #6, #4, #5
                 , "threshold": 0.83  ## default is 0.8, paper mentions 0.99 as thresh
                     })

In [7]:
additional_target_cui_terms_list = config['additional_target_cui_terms_list']
print("TARGET_NAME (For Entity-KG linking", config["TARGET_NAME"])
# entity = get_multiple_nel(TARGET_NAME,nlp=nlp) # new func
entity = test_nel(config["TARGET_NAME"],nlp=nlp)
# #### Target terms - may manually change
## list of extracted CUIs meaning sepsis. Note that not all correct even here and with threshhold
list_target_cuis = [i[0] for i in entity._.kb_ents]
if len(additional_target_cui_terms_list) > 0:
    list_target_cuis = list(set(list_target_cuis + additional_target_cui_terms_list))
list_target_cuis

TARGET_NAME (For Entity-KG linking Retinal Vein Occlusion
Retinal Vein Occlusion C0035328


/opt/anaconda3/envs/Medrag/lib/python3.11/site-packages/scispacy/abbreviation.py:248: UserWarning: [W036] The component 'matcher' does not have any patterns defined.
  global_matches = self.global_matcher(doc)


['C0035328', 'C0154841', 'C0339505']

In [8]:
## randomly sample, from CUIs that are not too rare? or just random sample
df_cands = df_kg.loc[~((df_kg["SUBJECT_CUI"].isin(list_target_cuis)) | (df_kg["OBJECT_CUI"].isin(list_target_cuis)))].drop_duplicates(subset="SUBJECT_CUI")
df_cands = df_cands.sample(800)
df_cands["cui"] = df_cands["SUBJECT_CUI"]
df_cands

,SUBJECT_CUI,SUBJECT_NAME,PREDICATE,OBJECT_CUI,OBJECT_NAME,first_year_pair,first_year_triple,cui
1319803,C5418188,belzutifan,TREATS,C0007134,Renal Cell Carcinoma,2021,2021,C5418188
3901512,C0339697,Congenital color blindness,PART_OF,C0027361,Persons,1964,1964,C0339697
1104875,C1265931,Chronic hypertrophy,PROCESS_OF,C0030705,Patients,1975,1975,C1265931
288223,C0079082,"Arthritis-Encephalitis Virus, Caprine",ISA,C0079679,Genus: Lentivirus group,1983,1983,C0079082
46219,C1281307,Entire central retinal artery,LOCATION_OF,C0028778,Obstruction,1919,1919,C1281307
...,...,...,...,...,...,...,...,...
21529,C0006266,Bronchial Spasm,PROCESS_OF,C0030705,Patients,1967,1967,C0006266
99617,C2975435,edoxaban,TREATS,C0030705,Patients,2019,2019,C2975435
61744,C0201594,Interferon gamma assay,USES,C0920508,enzyme linked immunospot assay,1997,1997,C0201594
592031,C0860742,Blood Eosinophilias,PROCESS_OF,C0030705,Patients,2020,2020,C0860742


In [9]:
df_cands["PREDICATE"].value_counts().head(20)

PREDICATE
PROCESS_OF         209
ISA                143
PART_OF             95
TREATS              89
LOCATION_OF         50
INTERACTS_WITH      36
ASSOCIATED_WITH     27
USES                23
CAUSES              21
ADMINISTERED_TO     18
INHIBITS            15
STIMULATES          11
AFFECTS             10
DISRUPTS             9
AUGMENTS             7
compared_with        7
COEXISTS_WITH        7
MEASURES             5
PREDISPOSES          5
DIAGNOSES            3
Name: count, dtype: int64

In [12]:
df_cands =  get_kg_connections(df_hits=df_cands, df_kg=df_kg, list_target_cuis=list_target_cuis, get_simplePathLengths =False)
df_cands = df_cands.loc[(df_cands["shortest_path_length"].between(2,99)) & (df_cands["node_degree"]>1)].sort_values(by=["shortest_path_length","node_degree"],ascending=False)
df_cands = df_cands.merge(df_kg[["SUBJECT_CUI","SUBJECT_NAME"]].drop_duplicates("SUBJECT_CUI"),left_on=["cui"],right_on="SUBJECT_CUI")
df_cands

Calculating Paths: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 800/800 [00:00<00:00, 18969.09it/s]


,cui,shortest_path_length,node_degree,SUBJECT_CUI,SUBJECT_NAME
0,C0020415,3,90,C0020415,Hymenoptera
1,C0600462,3,35,C0600462,Phanerochaete chrysosporium
2,C0330206,3,34,C0330206,Ginkgo biloba
3,C0330338,3,31,C0330338,Mirabilis
4,C2613267,3,26,C2613267,sporulation
...,...,...,...,...,...
676,C1018130,2,2,C1018130,Macaca thibetana
677,C0401067,2,2,C0401067,Unilateral inguinal hernia NOS
678,C0043108,2,2,C0043108,WELL BABY VISIT
679,C0235983,2,2,C0235983,Normochromic anemia


In [13]:
df_cands.loc[df_cands["node_degree"]>2].head(30)

,cui,shortest_path_length,node_degree,SUBJECT_CUI,SUBJECT_NAME
0,C0020415,3,90,C0020415,Hymenoptera
1,C0600462,3,35,C0600462,Phanerochaete chrysosporium
2,C0330206,3,34,C0330206,Ginkgo biloba
3,C0330338,3,31,C0330338,Mirabilis
4,C2613267,3,26,C2613267,sporulation
5,C0063245,3,23,C0063245,hypusine
6,C1277793,3,20,C1277793,Specific antibody measurement
7,C0035186,3,17,C0035186,Residential Facilities
8,C0004913,3,17,C0004913,Bedbugs
9,C0598245,3,16,C0598245,Cruciferous Vegetable


In [5]:
df_tui = pd.read_csv("UMLS_SemanticTypes.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'UMLS_SemanticTypes.csv'